# Day Ten

## Task

The [task](https://adventofcode.com/2017/day/10) today is to calculate the hash of a list where the hash is defined as some knot-tying function.

### Part One

First, let's define a function that allows us to *twist* the knot according to the rules defined.

In [1]:
from collections import deque
from copy import copy
from itertools import islice


def twist(sequence, position, skip, length):
    s = copy(sequence)
    
    s.rotate(-position)
    s = deque(
        list(reversed(list(islice(s, 0, length))))
        + list(islice(s, length, len(s)))
    )
    s.rotate(position)
    
    return s

We can then define a `hash_` function that twists as per the task.

In [2]:
def hash_(sequence, lengths):
    s = deque(sequence)
    position = 0
    skip = 0
    
    for length in lengths:
        s = twist(s, position, skip, length)
        position = (position + skip + length) % len(s)
        skip += 1
    
    return list(s)

Finally, we can validate that the hashed sequence as as expected.

In [3]:
sequence = [0, 1, 2, 3, 4]
hashed_sequence = hash_(sequence, [3, 4, 1, 5])

assert hashed_sequence == [3, 4, 2, 1, 0]
assert hashed_sequence[0] * hashed_sequence[1] == 12

Against the puzzle, we find

In [4]:
with open('../../data/day10.txt') as f:
    data = f.read()
    
hashed_sequence = hash_(sequence=range(0, 256), lengths=[int(x) for x in data.split(',')])
hashed_sequence[0] * hashed_sequence[1]

11413

### Part Two

The second part of the task builds upon that which we did in the first. However, there is a bunch of pre- and post-processing to do. Let's begin by looking at the lengths.

According to the task, our input should not be a list of numbers but a string of bytes. That is, we should encode our list of numbers.

The method of encoding is not entirely clear to me. The task explicitly says that if given lengths `1,2,3`, we can expect a return of `49,44,50,44,51,17,31,73,47,23`. I assume then that for two digit lengths, we encode each digit separately. For example, `10,2,3` would become `49,48,44,50,44,51,17,31,73,47,23`.

In [5]:
def encode(string):
    return ','.join([str(ord(x)) for x in string] + ['17', '31', '73', '47', '23'])

assert encode('1,2,3') == '49,44,50,44,51,17,31,73,47,23'
assert encode('10,2,3') == '49,48,44,50,44,51,17,31,73,47,23'

Now, we need to extend our `hash` function to allow us to carry over the value of position and skip. We do this as follows.

In [6]:
def hash_(sequence, lengths, position=0, skip=0):
    s = deque(sequence)
    position_ = position
    skip_ = skip
    
    for length in lengths:
        s = twist(s, position_, skip_, length)
        position_ = (position_ + skip_ + length) % len(s)
        skip_ += 1
    
    return list(s), position_, skip_

Now, we can define an outer hash function that runs the hash cycles.

In [7]:
def cycle(sequence, lengths):
    s = copy(sequence)
    position = 0
    skip = 0
    
    for _ in range(0, 64):
        s, position, skip = hash_(s, lengths, position, skip)

    return s

We can then compute the sparse hash, dense hash and final hashed value as follows.

In [8]:
from functools import reduce

def output(lengths):
    sequence = range(0, 256)
    
    lengths = [int(x) for x in encode(lengths.strip()).split(',')]
    sparse_hash = cycle(range(0, 256), lengths)
    dense_hash = map(
        lambda x: reduce(lambda y, z: y ^ z, x), 
        [sparse_hash[x:x+16] for x in range(0, len(sparse_hash), 16)]
    )

    return ''.join([hex(x)[2:].zfill(2) for x in dense_hash])


assert output('') == 'a2582a3a0e66e6e86e3812dcb672a272'
assert output('AoC 2017') == '33efeb34ea91902bb2f59c9920caa6cd'
assert output('1,2,3') == '3efbe78a8d82f29979031a4aa0b16a9d'
assert output('1,2,4') == '63960835bcdc130f0b66d7ff4f6a5a8e'

Finally, on the puzzle, we find

In [10]:
with open('../../data/day10.txt') as f:
    data = f.read()

print(data)
output(data)

106,16,254,226,55,2,1,166,177,247,93,0,255,228,60,36



'7adfd64c2a03a4968cf708d1b7fd418d'